In [314]:
import h5py
import numpy as np

In [315]:
train_data = h5py.File("./data/train_catvnoncat.h5", "r")

In [316]:
test_data = h5py.File("./data/test_catvnoncat.h5", "r")

In [317]:
train_set_x = np.array(train_data["train_set_x"])
train_set_y = np.array(train_data["train_set_y"])
test_set_x = np.array(test_data["test_set_x"])
test_set_y = np.array(test_data["test_set_y"])

In [318]:
m_train = train_set_x.shape[0]
m_test = test_set_x.shape[0]

In [319]:
train_x = train_set_x.reshape(m_train, -1).T
test_x = test_set_x.reshape(m_test, -1).T
train_y = train_set_y.reshape(1, m_train)
test_y = test_set_y.reshape(1, m_test)

In [320]:
X, Y = train_x/255, train_y

In [321]:
def sigmoid(z):
    a = 1. / (1 + np.exp(-1.0*z))
    return a

In [346]:
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    Argument:
    dim --size of the w vector we want (or number of paremeters in this case)
    
    Result:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    w = np.zeros(shape=(dim, 1), dtype=np.float32)
    b = 0
    
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [330]:
def porpate(w, b, X, Y):
#     m = X.shape[1]
#     A = sigmoid(np.dot(w.T, X)+b)
#     cost = (-1. / m) * np.sum(Y*np.log(A) + (1-Y)*np.log(1-A), axis=1)
    
#     dw = (1.0/m)*np.dot(X, ((A-Y).T))
#     db = (1.0/m)*np.sum(A-Y, axis=1)
#     cost = np.squeeze(cost)
    
#     grads = {"dw":dw, "db":db}
#     return grads, cost
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) + b)
    cost = (-1. / m) * np.sum(Y*np.log(A) + (1-Y)*np.log(1-A), axis=1)
    
    dw = (1./m)*np.dot(X, ((A-Y).T))
    db = (1./m)*np.sum(((A-Y).T), axis=0)
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    #print("cost:", cost)
    
    grads = {"dw" : dw,
            "db" : db}
    return grads, cost

In [362]:
def optimize(X, Y, intertions, learning_rate):
    w = zeros((X.shape[0], 1))
    b = 0
    
    costs = []
    for i in range(intertions):
        grads, cost = porpate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        w = w - learning_rate*dw
        b = b - learning_rate*db
        
        if i % 100 == 0:
            costs.append(cost)
            print("Cost function %d is: %f" % (i, cost))
    params = {"w":w, "b":b}
    grads = {"dw":dw, "db":db}
    return params, grads, costs
    

In [363]:
# def predict(w, b, X):
#     m = X.shape[1]
#     Y_prediction = np.zeros((1, m))
#     A = sigmoid(np.dot(w.T, X) + b)
#     for i in range(A.shape[1]):
#         Y_prediction[0, i] = 1 if A[0, i] >=0.5 else 0
        
#     return Y_prediction

In [364]:
# params, grads, costs = optimizated(X, Y, 2000, 0.5)
# w = params["w"]
# b = params["b"]

In [365]:
def predict(w, b, X):
    """
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Argments:
    w -- weights, a numpy array of size (num_ps*num_px*3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    """
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        Y_prediction[0, i] = 1 if A[0, i] > 0.5 else 0
        
    assert(Y_prediction.shape == (1, m))
    return Y_prediction

In [366]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    w, b = initialize_with_zeros(X_train.shape[0])
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    print("w_after", w)
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [361]:
Y_prediction = predict(w, b, test_x/255)

In [156]:
print(100 - np.mean(np.abs(Y_prediction-test_y))*100)

34.0


In [177]:
print(test_y)

[[1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0 0
  0 1 0 0 1 1 1 0 0 0 1 1 1 0]]
